In [2]:
#%pip install requests-html requests pandas

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time
import csv

In [2]:
def BaseLinks(tag, totPageNum):
    base_url = 'https://stackoverflow.com/questions/tagged/'
    query_filter = "newest" #"Votes"
    pageLinks = []
    
    for pageNum in range (1, totPageNum+1):
        url = f"{base_url}{tag}?tab={query_filter}&page={pageNum}&pagesize=50"
        pageLinks.append(url)
    
    return pageLinks

In [3]:
#asdf = BaseLinks('pytorch', 384 )
#

In [3]:
def ParseQLinksFromBase(pageLinks):
    
    href = []
    qHeader = []
    
    for url in pageLinks:
        
        time.sleep(5)
        r = requests.get(url)
        so_soup = BeautifulSoup(r.text, 'html.parser')
        question_block = so_soup.find(id = "questions")
        if (question_block == None):
            print("Error in URL: "+ url)
        else:
            question_links = question_block.find_all("a", class_="s-link",href=True)
            for i in range(len(question_links)):
                qHeader.append(question_links[i].text)

        for qlink in question_links:
            href.append(qlink['href'])

    return href, qHeader


In [4]:
tagsToScrap = ['pytorch', 'torch', 'torchvision', 'pytorch-lightning', 'pytorch-dataloader', 'libtorch', 'pytorch-geometric', 'torchtext', 'torchscript', 'pytorch3d', 'gpytorch', 'torchaudio']
pageNumber = [386, 40, 11, 8, 7, 6, 4, 3, 2, 1, 1, 1]

In [5]:
pageLinks = []
for i in range(12):
     pageLinks.extend(BaseLinks(tagsToScrap[i], pageNumber[i]))

len(pageLinks)
#print(pageLinks)

470

In [ ]:
pytorchQLinks, QuesHeader = ParseQLinksFromBase(pageLinks)
len(pytorchQLinks)

In [7]:
qID = []
for i in range(len(pytorchQLinks)):
    qID.append(pytorchQLinks[i].split("/")[2])

NameError: name 'pytorchQLinks' is not defined

In [ ]:
len(qID)

23133

In [ ]:
QuesHeader[14]

"PyTorch: How to inprove pytorch's CPU utilization rate？"

In [ ]:
len(qID)

23133

In [ ]:
colHeader = ['QID', 'QLink','QHeader']
rows = []
with open('PostLinks.csv','w', encoding ='utf-8', newline='') as f:
    writer = csv.writer(f, delimiter="|")
    writer.writerow(colHeader)
    writer.writerows(zip(qID,pytorchQLinks,QuesHeader))

In [17]:
def getContentQuesData(url, soup):
    qID = url.split("/")[4]
    qHeader = soup.find(class_ = "question-hyperlink").text
    #mainbar = content_soup.find(id = "mainbar")
    qvote = soup.find(class_ = "question js-question").attrs.get('data-score')
    question = soup.find(class_ = "s-prose js-post-body").text
    qCodePart = soup.find(class_ = "s-prose js-post-body").findAll("code")
    qPostTime = soup.find(class_ = "post-signature owner flex--item").find('span').attrs.get('title')
    qAuthor = soup.find(class_ = "post-signature owner flex--item").find(class_ = "user-details").find(class_ = "d-none").text

    Rep = soup.find(class_ = "post-signature owner flex--item").find(class_ = "user-details").find(class_ = 'reputation-score')
    if(Rep != None):
        qAuthorRep = Rep.text
    else:
        qAuthorRep = 0
    #qComments = content_soup.find(class_ = ) comments are not collected
    numAnswers = soup.find(id = 'answers').find(class_ = "mb0").get("data-answercount")

    with open('Questions.csv','a+', encoding ='utf-8', newline='') as quesfile:
        writer = csv.writer(quesfile, delimiter="|")
        writer.writerow([qID, qHeader, qvote, question, qCodePart, qPostTime, qAuthor, qAuthorRep, numAnswers])

    return qID, numAnswers
    


In [26]:
def getContentAnsData(qid, numAns, soup):
    hasAcceptedAns = soup.find(class_ = "answer js-answer accepted-answer js-accepted-answer")
    acceptFlag = "False"
    accAns = accAnsCode = ""

    if (hasAcceptedAns != None):
        acceptFlag = "True"
        accAnsId = soup.find(class_ = "answer js-answer accepted-answer js-accepted-answer").attrs.get('data-answerid')
        accAnsVote = soup.find(class_ = "answer js-answer accepted-answer js-accepted-answer").attrs.get('data-score')
        accAns = soup.find(class_ = "answer js-answer accepted-answer js-accepted-answer").find(class_ = 's-prose js-post-body').text
        accAnsCode = soup.find(class_ = "answer js-answer accepted-answer js-accepted-answer").find(class_ = 's-prose js-post-body').findAll("code")
        numAns = int(numAns) 
        numAns -= 1

        with open('Answers.csv', 'a+', encoding ='utf-8',newline='') as ansfile:
            writer = csv.writer(ansfile, delimiter="|")
            writer.writerow([qid, acceptFlag, accAnsId, accAnsVote, accAns, accAnsCode])
        

    #allAnsIds = []
    #allAnswers = []
    #allAnswersCode = []
    answerBlock = soup.findAll(class_ = "answer js-answer")
    for i in range(len(answerBlock)):#int(numAns)):
        
        singleBlock = answerBlock[i]
        ansId = singleBlock.attrs.get("data-answerid")
        ansVote = singleBlock.attrs.get("data-score")
        #allAnsIds.append(ansId)
        ans = singleBlock.find(class_ = "s-prose js-post-body").text
        #allAnswers.append(ans)
        ansCode = singleBlock.find(class_ = "s-prose js-post-body").findAll("code")
        #allAnswersCode.append(ansCode)

        with open('Answers.csv', 'a+', encoding ='utf-8', newline='') as ansfile:
            writer = csv.writer(ansfile, delimiter="|")
            writer.writerow([qid, acceptFlag, ansId, ansVote, ans, ansCode])

In [20]:
def getContentData(contentUrl):
    r = requests.get(contentUrl)
    content_soup = BeautifulSoup(r.text, 'html.parser')
    qID, totAns = getContentQuesData(contentUrl,content_soup)
    getContentAnsData(qID, totAns, content_soup)
    return int(totAns)
 


In [21]:
#temp = getContentData('https://stackoverflow.com/questions/46833212/looping-through-elements-with-beautifulsoup')

In [24]:
totalAnswers = counter = 0
print(totalAnswers)
base = 'https://stackoverflow.com'
with open('PostLinks.csv', encoding ='utf-8', newline='') as postLinks:
    reader = csv.reader(postLinks, delimiter='|')
    for row in reader:
        if(row[0]=='QID'):
            continue
        link = row[1]
        contentParseLink = base + link
        numAnswerSingleLink = getContentData(contentParseLink)
        totalAnswers += numAnswerSingleLink
        counter += 1
        print(counter)
        print(totalAnswers)
        time.sleep(1.5)
#worked until 1216 line in postlinks copy



0
1
0
2
0
3
0
4
0
5
1
6
1
7
1
8
2
9
2
10
2
11
2
12
2
13
2
14
2
15
2
16
2
17
3
18
3
19
4
20
4
21
4
22
5
23
5
24
5
25
5
26
5
27
5
28
6
29
6
30
6
31
6
32
6
33
6
34
6
35
6
36
6
37
6
38
7
39
8
40
8
41
9
42
9
43
9
44
10
45
11
46
11
47
12
48
12
49
12
50
12
51
12
52
13
53
13
54
13
55
14
56
14
57
14
58
14
59
16
60
17
61
18
62
18
63
18
64
18
65
18
66
18
67
18
68
18
69
18
70
19
71
20
72
20
73
21
74
21
75
21
76
21
77
21
78
21
79
21
80
22
81
22
82
23
83
23
84
24
85
24
86
24
87
24
88
24
89
24
90
24
91
24
92
25
93
25
94
26
95
26
96
27
97
28
98
29
99
29
100
29
101
29
102
30
103
30
104
31
105
31
106
31
107
31
108
32
109
33
110
34
111
35
112
36
113
36
114
36
115
36
116
36
117
37
118
37
119
37
120
37
121
37
122
38
123
39
124
39
125
40
126
41
127
41
128
42
129
42
130
42
131
43
132
43
133
43
134
43
135
44
136
44
137
44
138
44
139
44
140
44
141
44
142
44
143
45
144
45
145
45
146
45
147
45
148
46
149
46
150
46
151
46
152
47
153
48
154
48
155
48
156
48
157
48
158
48
159
48
160
48
161
50
162
51
163
51
164
51
1

IndexError: list index out of range

In [25]:
#totalAnswers = counter = 0
print(totalAnswers)
base = 'https://stackoverflow.com'
with open('PostLinks.csv', encoding ='utf-8', newline='') as postLinks:
    reader = csv.reader(postLinks, delimiter='|')
    for row in reader:
        if(row[0]=='QID'):
            continue
        link = row[1]
        contentParseLink = base + link
        numAnswerSingleLink = getContentData(contentParseLink)
        totalAnswers += numAnswerSingleLink
        counter += 1
        print(counter)
        print(totalAnswers)
        time.sleep(1.5)
#worked until 4305 line in postlinks copy 2 # file got deleted



663
1215
663
1216
663
1217
664
1218
664
1219
664
1220
665
1221
666
1222
667
1223
669
1224
669
1225
670
1226
671
1227
671
1228
673
1229
674
1230
675
1231
676
1232
677
1233
678
1234
678
1235
678
1236
678
1237
679
1238
679
1239
680
1240
680
1241
681
1242
681
1243
682
1244
684
1245
685
1246
685
1247
685
1248
685
1249
685
1250
685
1251
685
1252
685
1253
687
1254
688
1255
689
1256
690
1257
691
1258
692
1259
693
1260
693
1261
694
1262
694
1263
695
1264
696
1265
697
1266
698
1267
699
1268
699
1269
700
1270
700
1271
701
1272
702
1273
702
1274
702
1275
703
1276
705
1277
706
1278
706
1279
706
1280
707
1281
707
1282
708
1283
708
1284
708
1285
708
1286
709
1287
710
1288
711
1289
711
1290
711
1291
713
1292
714
1293
715
1294
717
1295
718
1296
720
1297
721
1298
722
1299
722
1300
722
1301
723
1302
724
1303
724
1304
725
1305
726
1306
727
1307
728
1308
728
1309
729
1310
730
1311
731
1312
732
1313
733
1314
733
1315
734
1316
734
1317
735
1318
735
1319
736
1320
737
1321
738
1322
738
1323
739
1324
739
1325
7

IndexError: list index out of range

In [27]:
#totalAnswers = counter = 0
print(totalAnswers)
base = 'https://stackoverflow.com'
with open('PostLinks.csv', encoding ='utf-8', newline='') as postLinks:
    reader = csv.reader(postLinks, delimiter='|')
    for row in reader:
        if(row[0]=='QID'):
            continue
        link = row[1]
        contentParseLink = base + link
        numAnswerSingleLink = getContentData(contentParseLink)
        totalAnswers += numAnswerSingleLink
        counter += 1
        print(counter)
        print(totalAnswers)
        time.sleep(1.5)
#worked until 8560 line in postlinks copy 2 # file got deleted

3586
5518
3587
5519
3588
5520
3588
5521
3589
5522
3589
5523
3591
5524
3591
5525
3592
5526
3593
5527
3594
5528
3595
5529
3600
5530
3601
5531
3601
5532
3602
5533
3602
5534
3603
5535
3604
5536
3604
5537
3605
5538
3605
5539
3606
5540
3606
5541
3606
5542
3606
5543
3606
5544
3607
5545
3608
5546
3608
5547
3608
5548
3609
5549
3610
5550
3611
5551
3611
5552
3612
5553
3614
5554
3614
5555
3615
5556
3616
5557
3616
5558
3618
5559
3620
5560
3620
5561
3621
5562
3622
5563
3623
5564
3623
5565
3624
5566
3624
5567
3625
5568
3625
5569
3627
5570
3627
5571
3628
5572
3629
5573
3630
5574
3632
5575
3632
5576
3632
5577
3633
5578
3634
5579
3634
5580
3635
5581
3636
5582
3637
5583
3638
5584
3639
5585
3639
5586
3639
5587
3640
5588
3641
5589
3643
5590
3643
5591
3643
5592
3643
5593
3644
5594
3645
5595
3645
5596
3646
5597
3646
5598
3646
5599
3647
5600
3647
5601
3648
5602
3648
5603
3648
5604
3648
5605
3649
5606
3649
5607
3649
5608
3650
5609
3651
5610
3651
5611
3652
5612
3653
5613
3653
5614
3654
5615
3655
5616
3656
5617


AttributeError: 'NoneType' object has no attribute 'text'

In [29]:
#totalAnswers = counter = 0
print(totalAnswers)
base = 'https://stackoverflow.com'
with open('PostLinks.csv', encoding ='utf-8', newline='') as postLinks:
    reader = csv.reader(postLinks, delimiter='|')
    for row in reader:
        if(row[0]=='QID'):
            continue
        link = row[1]
        contentParseLink = base + link
        numAnswerSingleLink = getContentData(contentParseLink)
        totalAnswers += numAnswerSingleLink
        counter += 1
        print(counter)
        print(totalAnswers)
        time.sleep(1.5)
#worked until 8560 line in postlinks copy 2 # file got deleted

11974
14077
11975
14078
11976
14079
11978
14080
11978
14081
11979
14082
11980
14083
11981
14084
11982
14085
11983
14086
11984
14087
11985
14088
11989
14089
11990
14090
11990
14091
11991
14092
11995
14093
11996
14094
11997
14095
11997
14096
11998
14097
11999
14098
12001
14099
12002
14100
12002
14101
12002
14102
12003
14103
12005
14104
12006
14105
12007
14106
12008
14107
12009
14108
12011
14109
12015
14110
12016
14111
12017
14112
12017
14113
12018
14114
12019
14115
12021
14116
12021
14117
12022
14118
12023
14119
12024
14120
12025
14121
12026
14122
12028
14123
12029
14124
12030
14125
12031
14126
12031
14127
12032
14128
12033
14129
12034
14130
12034
14131
12034
14132
12036
14133
12039
14134
12040
14135
12041
14136
12044
14137
12046
14138
12048
14139
12048
14140
12049
14141
12049
14142
12050
14143
12051
14144
12051
14145
12052
14146
12053
14147
12053
14148
12054
14149
12055
14150
12056
14151
12056
14152
12056
14153
12057
14154
12058
14155
12058
14156
12058
14157
12059
14158
12060
14159
1206

AttributeError: 'NoneType' object has no attribute 'text'

In [30]:
#totalAnswers = counter = 0
print(totalAnswers)
base = 'https://stackoverflow.com'
with open('PostLinks.csv', encoding ='utf-8', newline='') as postLinks:
    reader = csv.reader(postLinks, delimiter='|')
    for row in reader:
        if(row[0]=='QID'):
            continue
        link = row[1]
        contentParseLink = base + link
        numAnswerSingleLink = getContentData(contentParseLink)
        totalAnswers += numAnswerSingleLink
        counter += 1
        print(counter)
        print(totalAnswers)
        time.sleep(1.5)
#worked until 5551 line in postlinks copy 3 

18878
19627
18879
19628
18879
19629
18880
19630
18881
19631
18882
19632
18883
19633
18884
19634
18884
19635
18885
19636
18885
19637
18886
19638
18888
19639
18889
19640
18889
19641
18890
19642
18890
19643
18891
19644
18891
19645
18891
19646
18891
19647
18892
19648
18894
19649
18894
19650
18895
19651
18897
19652
18898
19653
18898
19654
18898
19655
18898
19656
18899
19657
18900
19658
18900
19659
18901
19660
18902
19661
18903
19662
18904
19663
18904
19664
18905
19665
18905
19666
18906
19667
18907
19668
18907
19669
18908
19670
18909
19671
18909
19672
18910
19673
18910
19674
18911
19675
18911
19676
18911
19677
18913
19678
18918
19679
18920
19680
18921
19681
18922
19682
18923
19683
18923
19684
18924
19685
18926
19686
18926
19687
18926
19688
18927
19689
18928
19690
18929
19691
18930
19692
18931
19693
18932
19694
18932
19695
18934
19696
18935
19697
18936
19698
18937
19699
18938
19700
18939
19701
18941
19702
18941
19703
18943
19704
18943
19705
18945
19706
18946
19707
18947
19708
18949
19709
1895

In [31]:
numOfZeros = 0
with open('Questions.csv', encoding ='utf-8', newline='') as csv_file:
    reader = csv.reader(csv_file, delimiter='|')
    for row in reader:
        if(row[8]=='0'):
            numOfZeros+=1
        
print("Number of Questions with No answers :",numOfZeros)

Number of Questions with No answers : 6588


In [33]:
numOfAccept = 0
with open('Answers.csv', encoding ='utf-8', newline='') as csv_file:
    reader = csv.reader(csv_file, delimiter='|')
    for row in reader:
        if(row[1]=='True'):
            numOfAccept+=1
        
print("Number of Questions with Accepted answers :",numOfAccept)

Number of Questions with Accepted answers : 12179


In [2]:
import csv 
with open('Questions.csv', encoding ='utf-8', newline='') as csv_file:
    reader = csv.reader(csv_file, delimiter='|')
    for row in reader:
        print(row[0], row[1])

74250490 How to calculate mAP on test dataset using pytorch?
74249708 Pytorch least squares residuals computation
74249550 How does one find the name of a local variable that is a lambda function in a method is not being able to be pickled?
74248377 What is an efficient way to make a dataset and dataloader for high frequency time series with multiple individuals?
74248099 Tensor repeat for image patches
74247049 Round Tensor in Python but preserve the decimal places
74246129 Most efficient way to store large amounts of pytorch tensors in disk
74245656 how to use pretrained model CNN with real images various size?
74244179 Input type (torch.cuda.ByteTensor) and weight type (torch.cuda.FloatTensor) should be the same
74243999 when I use the trtexec --onnx=** --saveEngine=** to transfer my onnx to a trt model, I got a warning from TensorRT
74243749 How does pytorch differential grid_sample?
74243417 concatenate image and mask and make a 4-channel image and save image with 4 channels and r